# Inland / Pluvial + Riverine Flood Risk — Interactive Demo

This notebook walks through the full analysis pipeline step by step.  
Run cells in order. All outputs are written to `/project/output/`.

> **Key insight:** Saturation, recovery windows, and system margins matter more than peak rainfall alone.

In [ ]:
import sys
from pathlib import Path

# Add src to path
PROJECT = Path.cwd().parent
sys.path.insert(0, str(PROJECT / 'src'))

from data_loader import load_csv
from metrics import compute_all_metrics
from risk_states import assign_risk_states, summary_table
from visualization import plot_all_panels

import matplotlib
matplotlib.rcParams['figure.dpi'] = 120
print('Imports OK')

## 1. Load a Dataset

Switch `DATASET` to `'ahr'`, `'future'`, or point `CSV_PATH` at your own file.

In [ ]:
DATASET = 'midwest'   # 'midwest' | 'ahr' | 'future'

PATHS = {
    'midwest': PROJECT / 'data' / 'sample_midwest_flood.csv',
    'ahr':     PROJECT / 'data' / 'sample_europe_2021_ahr.csv',
    'future':  PROJECT / 'data' / 'sample_future_intensified_precip.csv',
}

CSV_PATH = PATHS[DATASET]
hourly = load_csv(CSV_PATH)
hourly.head()

## 2. Compute Metrics

Runs EFD, CFL, PSe, and all streak counters.

In [ ]:
hourly, daily = compute_all_metrics(hourly)

print('Hourly columns:', list(hourly.columns))
print('Daily  columns:', list(daily.columns))
hourly[['rainfall_mm','soil_moisture','EFD','CFL_hour']].describe()

## 3. Assign Risk States & Multiplier

In [ ]:
daily = assign_risk_states(daily)

tbl = summary_table(daily)
tbl

## 4. Risk State Distribution

In [ ]:
import matplotlib.pyplot as plt
from risk_states import STATE_COLORS

counts = daily['risk_state'].value_counts()
colors = [STATE_COLORS.get(s, '#ccc') for s in counts.index]

fig, ax = plt.subplots(figsize=(5, 3))
ax.bar(counts.index, counts.values, color=colors, edgecolor='white')
ax.set_title('Days per Risk State')
ax.set_ylabel('Days')
plt.tight_layout()
plt.show()

## 5. Full 5-Panel Figure

In [ ]:
out = plot_all_panels(
    hourly=hourly,
    daily=daily,
    title_prefix=DATASET.title(),
    output_dir=PROJECT / 'output',
    filename=f'{DATASET}_notebook_demo.png',
)

# Display inline
from IPython.display import Image
Image(str(out))

## 6. Risk Multiplier Deep Dive

The multiplier encodes nonlinear escalation from compound antecedent stress.

In [ ]:
fig, axes = plt.subplots(3, 1, figsize=(13, 8), sharex=True)

dates = daily.index.to_pydatetime()

axes[0].bar(dates, daily['daily_CFL'], color='#ff7f0e', alpha=0.7)
axes[0].axhline(80,  color='r', ls='--', lw=0.8, label='Straining (80)')
axes[0].axhline(160, color='darkred', ls='--', lw=0.8, label='Failure (160)')
axes[0].set_ylabel('daily CFL'); axes[0].legend(fontsize=7)

axes[1].bar(dates, daily['daily_PSe'], color='#9467bd', alpha=0.7)
axes[1].axhline(4, color='r', ls='--', lw=0.8, label='Straining (4)')
axes[1].axhline(8, color='darkred', ls='--', lw=0.8, label='Failure (8)')
axes[1].set_ylabel('daily PSe'); axes[1].legend(fontsize=7)

axes[2].plot(dates, daily['risk_multiplier'], color='#2ca02c', lw=2)
axes[2].fill_between(dates, 1, daily['risk_multiplier'], alpha=0.15, color='#2ca02c')
axes[2].axhline(1, color='grey', ls='--', lw=0.8)
axes[2].set_ylabel('Risk Multiplier')

for ax in axes:
    ax.grid(True, alpha=0.3)

fig.suptitle(f'{DATASET.title()} — CFL, PSe, and Risk Multiplier', fontweight='bold')
plt.tight_layout()
plt.show()

print(f'Peak risk multiplier: {daily["risk_multiplier"].max():.2f}x')
print(f'Peak daily CFL:       {daily["daily_CFL"].max():.1f}')
print(f'Peak daily PSe:       {daily["daily_PSe"].max():.3f}')

## 7. Run Your Own CSV

```python
my_hourly = load_csv('path/to/my_data.csv')
my_hourly, my_daily = compute_all_metrics(my_hourly)
my_daily = assign_risk_states(my_daily)
print(summary_table(my_daily))
plot_all_panels(my_hourly, my_daily, title_prefix='My Event',
                output_dir=PROJECT / 'output', filename='my_event.png')
```